# Notebook 07: Évaluation du Système et Logging avec Weights & Biases

Ce notebook montre comment utiliser nos modules d'évaluation (`RagEvaluator`, `SynthesisEvaluator`) pour mesurer la performance de "Cognitive Swarm" et comment intégrer ces résultats avec Weights & Biases (W&B) en utilisant `WandBMetricsLogger`.

**Prérequis :**
* Environnement configuré (`00_setup_environment.ipynb`), y compris les clés API pour OpenAI et W&B dans `.env`.
* MongoDB accessible et potentiellement peuplé avec des données via `01_data_ingestion_and_embedding.ipynb` (pour que l'évaluation RAG soit significative).
* Les bibliothèques `wandb` et `pandas` doivent être installées (elles le sont si `environment.yml` a été utilisé).
* Avoir créé des jeux de données d'évaluation (ou utiliser les exemples par défaut/simplifiés) :
    * `rag_eval_dataset.json` pour `RagEvaluator`.
    * Un jeu de données pour `SynthesisEvaluator` contenant des triplets `(query, context, synthesis_to_evaluate)`.

**Étapes :**
1. Configuration initiale (imports, logging, préparation W&B).
2. Démonstration de l'évaluation RAG avec logging W&B.
3. Démonstration de l'évaluation de la Synthèse avec logging W&B.
4. Discussion sur l'utilisation de `scripts/run_evaluation.py` pour une orchestration complète.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio
import pandas as pd # Pour créer des tables pour W&B
from typing import Optional, List, Dict, Any # Ajout de List, Dict, Any

# Ajout de la racine du projet au PYTHONPATH
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer les modules d'évaluation et le logger W&B
from src.rag.retrieval_engine import RetrievalEngine # Nécessaire pour RagEvaluator
from src.evaluation.rag_evaluator import RagEvaluator, RagEvaluationMetrics
from src.evaluation.synthesis_evaluator import SynthesisEvaluator, SynthesisEvaluationResult, EvaluationAspectScore # Ajout de EvaluationAspectScore
# L'import de SynthesisEvalItem n'est pas utilisé directement ici, mais est défini dans le script run_evaluation.py
from src.evaluation.metrics_logger import WandBMetricsLogger
from src.vector_store.mongodb_manager import MongoDBManager # Pour les noms de collection par défaut

setup_logging(level="INFO")
logger = logging.getLogger("nb_07_evaluation_logging")

# --- Vérifications critiques pour ce notebook ---
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
if active_embedding_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error("ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée. L'évaluation RAG (via RetrievalEngine) échouera.")
elif active_embedding_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error("ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée. L'évaluation RAG échouera.")

active_llm_judge_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
if active_llm_judge_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error("ERREUR : Le fournisseur de LLM (pour le juge de synthèse) est 'openai', mais OPENAI_API_KEY n'est pas configurée. L'évaluation de synthèse échouera.")
elif active_llm_judge_provider == "huggingface_api" and not settings.HUGGINGFACE_API_KEY:
    logger.error("ERREUR : Le fournisseur de LLM (pour le juge de synthèse) est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée. L'évaluation de synthèse échouera.")
elif active_llm_judge_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error("ERREUR : Le fournisseur de LLM (pour le juge de synthèse) est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée. L'évaluation de synthèse échouera.")

if not settings.WANDB_API_KEY and not os.environ.get("WANDB_API_KEY"):
    logger.warning("WANDB_API_KEY non trouvé. Le logging sur W&B pourrait échouer ou demander une authentification interactive.")
if not settings.MONGO_URI:
    logger.error("MONGO_URI non configuré. RetrievalEngine ne pourra pas fonctionner pour l'évaluation RAG.")

# Nom de la collection pour les tests RAG (doit correspondre à ce qui a été ingéré)
COLLECTION_NAME_EVAL = getattr(settings, 'COLLECTION_NAME_NOTEBOOK', MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME)

### 1. Connexion à Weights & Biases

Avant de commencer les évaluations, nous allons nous assurer que nous pouvons nous connecter à W&B. Le `WandBMetricsLogger` s'en chargera. Si vous n'êtes pas déjà connecté via le CLI (`wandb login`), la présence de `WANDB_API_KEY` dans votre `.env` est fortement recommandée.

In [ ]:
async def run_rag_evaluation_notebook_demo(top_k_eval=3):
    logger.info(f"--- Début de l'Évaluation RAG (k={top_k_eval}) ---")
    
    # Vérification des prérequis pour le fournisseur d'embedding actif
    active_embedding_provider_check = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
    can_proceed = True
    if active_embedding_provider_check == "openai" and not settings.OPENAI_API_KEY:
        logger.error("RAG EVAL ERROR: OpenAI API Key manquant pour le fournisseur d'embedding OpenAI.")
        can_proceed = False
    elif active_embedding_provider_check == "ollama" and not settings.OLLAMA_BASE_URL:
        logger.error("RAG EVAL ERROR: OLLAMA_BASE_URL manquant pour le fournisseur d'embedding Ollama.")
        can_proceed = False
    if not settings.MONGO_URI: # Nécessaire pour RetrievalEngine
        logger.error("RAG EVAL ERROR: MONGO_URI manquant.")
        can_proceed = False
    if not can_proceed:
        return

    # Configuration pour W&B
    config_for_wandb_rag = {
        "evaluation_type": "RAG_Notebook_Demo",
        "retrieval_top_k_eval": top_k_eval,
        "rag_dataset_path": str(settings.EVALUATION_DATASET_PATH) if settings.EVALUATION_DATASET_PATH else "Internal Default",
        "mongo_collection": COLLECTION_NAME_EVAL, # Défini dans la cellule précédente
        "active_embedding_provider": active_embedding_provider_check
    }
    if active_embedding_provider_check == "openai":
        config_for_wandb_rag["active_embedding_model"] = settings.OPENAI_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.OPENAI_EMBEDDING_DIMENSION
    elif active_embedding_provider_check == "huggingface":
        config_for_wandb_rag["active_embedding_model"] = settings.HUGGINGFACE_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION
    elif active_embedding_provider_check == "ollama":
        config_for_wandb_rag["active_embedding_model"] = settings.OLLAMA_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.OLLAMA_EMBEDDING_MODEL_DIMENSION

    wb_logger = WandBMetricsLogger(
        project_name="CognitiveSwarm-Evaluations-Notebook",
        run_name=f"RAG_Eval_Demo_k{top_k_eval}_{active_embedding_provider_check}",
        config_to_log=config_for_wandb_rag,
        tags=["rag", "notebook_demo", f"k{top_k_eval}", active_embedding_provider_check]
    )

    if wb_logger.is_disabled:
        logger.warning("Logging W&B désactivé. Les métriques ne seront pas envoyées à W&B.")
    else:
        wb_logger.start_run()

    retrieval_engine_instance_rag = None
    try:
        retrieval_engine_instance_rag = RetrievalEngine(collection_name=COLLECTION_NAME_EVAL)
    except Exception as e:
        logger.error(f"Échec d'initialisation du RetrievalEngine : {e}", exc_info=True)
        if wb_logger and not wb_logger.is_disabled and wb_logger.wandb_run: wb_logger.end_run(exit_code=1)
        return

    rag_eval_dataset_path = Path(settings.EVALUATION_DATASET_PATH) if settings.EVALUATION_DATASET_PATH else None
    # RagEvaluator utilise son propre dataset par défaut si rag_eval_dataset_path est None ou si le fichier n'est pas trouvé
    evaluator_rag = RagEvaluator(
        retrieval_engine=retrieval_engine_instance_rag,
        eval_dataset_path=rag_eval_dataset_path 
    )
    
    # Mettre à jour la config W&B avec le chemin réel utilisé par l'évaluateur si différent
    if wb_logger and not wb_logger.is_disabled and wb_logger.wandb_run:
        actual_dataset_path_logged = str(evaluator_rag.eval_dataset_path) if evaluator_rag.eval_dataset_path else "Internal Default"
        if config_for_wandb_rag.get("rag_dataset_path") != actual_dataset_path_logged:
            wb_logger.log_configuration({"rag_dataset_path_actual": actual_dataset_path_logged})


    if not evaluator_rag.eval_dataset:
        logger.error("Jeu de données d'évaluation RAG vide. Arrêt.")
        if wb_logger and not wb_logger.is_disabled and wb_logger.wandb_run: wb_logger.end_run(exit_code=1)
        return

    logger.info(f"Évaluation RAG avec {len(evaluator_rag.eval_dataset)} requêtes (Dataset: {actual_dataset_path_logged}).")
    rag_metrics = evaluator_rag.evaluate(k=top_k_eval)

    if rag_metrics:
        evaluator_rag.print_results(rag_metrics)
        if wb_logger and not wb_logger.is_disabled and wb_logger.wandb_run:
            wb_logger.log_rag_evaluation_results(rag_metrics, eval_name=f"RAG_Demo_Eval_k{top_k_eval}")
            
            eval_details_list = []
            for item in evaluator_rag.eval_dataset:
                eval_details_list.append({
                    "query_id": item["query_id"],
                    "query_text": item["query_text"],
                    "expected_relevant_chunk_ids": ", ".join(item["expected_relevant_chunk_ids"])
                })
            if eval_details_list:
                try:
                    details_df = pd.DataFrame(eval_details_list)
                    wb_logger.log_dataframe_as_table(details_df, "RAG_Evaluation_Queries_Used")
                except ImportError: # pandas est importé au début, mais bonne pratique de garder
                    logger.warning("Pandas n'est pas installé, impossible de logger la table de détails RAG.")
                except Exception as e_df:
                    logger.error(f"Erreur lors du logging de la table de détails RAG : {e_df}")
    else:
        logger.warning("L'évaluation RAG n'a pas produit de métriques.")

    if wb_logger and not wb_logger.is_disabled and wb_logger.wandb_run:
        wb_logger.end_run()

# Exécuter la démo d'évaluation RAG
asyncio.run(run_rag_evaluation_notebook_demo(top_k_eval=3))

### 2. Démonstration de l'Évaluation RAG

Nous allons instancier `RetrievalEngine` et `RagEvaluator`, exécuter une évaluation sur un petit jeu de données (le jeu par défaut de `RagEvaluator` ou un fichier JSON si configuré), puis logger les résultats et la configuration sur W&B.

**Action Requise :** Pour des résultats significatifs, assurez-vous d'avoir un fichier `rag_eval_dataset.json` (chemin configurable via `settings.EVALUATION_DATASET_PATH`) contenant des requêtes et les `chunk_id` pertinents attendus pour votre corpus. Le `RagEvaluator` utilisera son jeu de données de démo interne si ce fichier n'est pas trouvé ou non spécifié.

In [ ]:
async def run_synthesis_evaluation_notebook_demo(judge_model_name_override: Optional[str] = None):
    # Déterminer le provider et le nom du modèle LLM Juge qui sera utilisé
    # SynthesisEvaluator utilise get_llm, qui prendra DEFAULT_LLM_MODEL_PROVIDER par défaut
    actual_judge_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
    actual_judge_model_name = judge_model_name_override # L'override du nom du modèle pour le provider par défaut
    
    if not actual_judge_model_name: # Si pas d'override, prendre le modèle par défaut du provider
        if actual_judge_provider == "openai":
            actual_judge_model_name = settings.DEFAULT_OPENAI_GENERATIVE_MODEL
        elif actual_judge_provider == "huggingface_api":
            actual_judge_model_name = settings.HUGGINGFACE_REPO_ID
        elif actual_judge_provider == "ollama":
            actual_judge_model_name = settings.OLLAMA_GENERATIVE_MODEL_NAME
        else:
            actual_judge_model_name = "Unknown_Default_Model"
            logger.warning(f"Provider LLM Juge inconnu: {actual_judge_provider}. Le nom du modèle pourrait être incorrect.")
            
    logger.info(f"--- Début de l'Évaluation de la Synthèse (Juge Provider: {actual_judge_provider}, Juge Model: {actual_judge_model_name}) ---")

    # Vérification des prérequis pour le LLM Juge
    can_proceed = True
    if actual_judge_provider == "openai" and not settings.OPENAI_API_KEY:
        logger.error("SYNTHESIS EVAL ERROR: OpenAI API Key manquant pour le LLM juge OpenAI.")
        can_proceed = False
    elif actual_judge_provider == "huggingface_api" and not settings.HUGGINGFACE_API_KEY:
        logger.error("SYNTHESIS EVAL ERROR: HuggingFace API Key manquant pour le LLM juge HuggingFace API.")
        can_proceed = False
    elif actual_judge_provider == "ollama":
        if not settings.OLLAMA_BASE_URL:
            logger.error("SYNTHESIS EVAL ERROR: OLLAMA_BASE_URL manquant pour le LLM juge Ollama.")
            can_proceed = False
        if not settings.OLLAMA_GENERATIVE_MODEL_NAME and not judge_model_name_override: # Si aucun nom de modèle n'est défini pour Ollama
             logger.error(f"SYNTHESIS EVAL ERROR: Nom du modèle Ollama pour le juge non défini (OLLAMA_GENERATIVE_MODEL_NAME ou override).")
             can_proceed = False
    if not can_proceed:
        return

    # --- PRÉPARER VOS DONNÉES DE TEST ICI ---
    sample_query_synth = "What are the main challenges in robotic grasping using reinforcement learning?"
    sample_context_synth = """
Challenge 1: Sample Inefficiency. RL algorithms often require a vast amount of data (trials) to learn effective grasping policies. This is costly and time-consuming on physical robots.
Challenge 2: Sim-to-Real Gap. Models trained in simulation may not transfer well to real robots due to differences in dynamics, sensing, and appearance.
Challenge 3: Reward Design. Crafting appropriate reward functions that guide the agent towards successful and robust grasping without unintended behaviors is difficult.
Challenge 4: High-Dimensional State/Action Spaces. Grasping involves continuous and high-dimensional inputs (e.g., camera images) and outputs (e.g., robot joint commands).
(Source: Fictional summary based on general knowledge for demo purposes)
"""
    sample_synthesis_to_eval = """
Robotic grasping using reinforcement learning faces several key challenges. Firstly, sample inefficiency means many trials are needed.
Secondly, bridging the sim-to-real gap is problematic due to mismatches between simulation and reality.
Thirdly, designing effective reward functions is complex. Lastly, the high dimensionality of state and action spaces poses difficulties.
These challenges are actively being researched.
"""
    # -----------------------------------------

    config_for_wandb_synth = {
        "evaluation_type": "Synthesis_Notebook_Demo",
        "original_query_snippet": sample_query_synth[:100] + "..." if len(sample_query_synth) > 100 else sample_query_synth,
        "judge_llm_provider": actual_judge_provider,
        "judge_llm_model_used": actual_judge_model_name
    }

    wb_logger_synth = WandBMetricsLogger(
        project_name="CognitiveSwarm-Evaluations-Notebook",
        run_name=f"Synth_Eval_Demo_Judge_{actual_judge_provider}_{str(actual_judge_model_name).split('/')[-1].replace('.', '_')}",
        config_to_log=config_for_wandb_synth,
        tags=["synthesis", "notebook_demo", "llm_as_judge", actual_judge_provider]
    )
    if wb_logger_synth.is_disabled:
        logger.warning("Logging W&B désactivé. Les métriques de synthèse ne seront pas envoyées à W&B.")
    else:
        wb_logger_synth.start_run()
    
    evaluator_synth = None
    try:
        evaluator_synth = SynthesisEvaluator(
            judge_llm_provider=None, 
            judge_llm_model_name=judge_model_name_override 
        )
    except Exception as e:
        logger.error(f"Échec d'initialisation du SynthesisEvaluator: {e}", exc_info=True)
        if wb_logger_synth and not wb_logger_synth.is_disabled and wb_logger_synth.wandb_run: wb_logger_synth.end_run(exit_code=1)
        return

    logger.info(f"Évaluation de la synthèse pour la requête: '{sample_query_synth[:50]}...'")
    synthesis_metrics: Optional[SynthesisEvaluationResult] = await evaluator_synth.evaluate_synthesis(
        query=sample_query_synth,
        synthesis=sample_synthesis_to_eval,
        context=sample_context_synth
    )

    if synthesis_metrics:
        evaluator_synth.print_results(synthesis_metrics, query=sample_query_synth)
        if wb_logger_synth and not wb_logger_synth.is_disabled and wb_logger_synth.wandb_run:
            wb_logger_synth.log_synthesis_evaluation_results(synthesis_metrics, eval_name="Synthesis_Demo_Eval")
            
            synth_detail_data = [{
                "query": sample_query_synth,
                "context_snippet": sample_context_synth[:500] + "..." if len(sample_context_synth) > 500 else sample_context_synth,
                "evaluated_synthesis": sample_synthesis_to_eval,
                "relevance_score": synthesis_metrics.get("relevance", {}).get("score") if synthesis_metrics.get("relevance") else None,
                "relevance_reasoning": synthesis_metrics.get("relevance", {}).get("reasoning") if synthesis_metrics.get("relevance") else None,
                "faithfulness_score": synthesis_metrics.get("faithfulness", {}).get("score") if synthesis_metrics.get("faithfulness") else None,
                "faithfulness_reasoning": synthesis_metrics.get("faithfulness", {}).get("reasoning") if synthesis_metrics.get("faithfulness") else None,
            }]
            try:
                details_df_synth = pd.DataFrame(synth_detail_data)
                wb_logger_synth.log_dataframe_as_table(details_df_synth, "Synthesis_Evaluation_Run_Detail")
            except ImportError: 
                logger.warning("Pandas n'est pas installé, impossible de logger la table de détails de synthèse.")
            except Exception as e_df_s:
                logger.error(f"Erreur lors du logging de la table de détails de synthèse: {e_df_s}")

    if wb_logger_synth and not wb_logger_synth.is_disabled and wb_logger_synth.wandb_run:
        wb_logger_synth.end_run()

# Exécuter la démo d'évaluation de synthèse
# Vous pouvez passer un nom de modèle pour surcharger le juge : par exemple, judge_model_name_override="gpt-4o"
# Si None, il utilisera le modèle génératif par défaut du provider configuré dans settings.py
asyncio.run(run_synthesis_evaluation_notebook_demo(judge_model_name_override=None))

### 3. Démonstration de l'Évaluation de la Synthèse

Nous allons maintenant évaluer la qualité d'une synthèse (pertinence, fidélité) en utilisant `SynthesisEvaluator` et un LLM comme juge.

**Action Requise :** Pour cette section, vous devez fournir :
1.  Une **requête utilisateur** (`sample_query_synth`).
2.  Un **contexte** (`sample_context_synth`) qui aurait été fourni à l'agent de synthèse pour générer la réponse.
3.  Une **synthèse à évaluer** (`sample_synthesis_to_eval`) qui est la sortie de votre `SynthesisAgent` pour cette requête et ce contexte.

Vous pouvez obtenir ces éléments en exécutant le notebook `04_langgraph_workflow_design.ipynb` (ou `scripts/run_cognitive_swarm.py`) pour une requête donnée et en copiant/collant la requête, le contexte (qui peut être une combinaison des messages pertinents ou des chunks récupérés) et la sortie de synthèse finale.

In [ ]:
async def run_synthesis_evaluation_notebook_demo(judge_model_override: Optional[str] = None):
    logger.info(f"--- Début de l'Évaluation de la Synthèse (Juge: {judge_model_override or settings.DEFAULT_OPENAI_MODEL}) ---")

    if not settings.OPENAI_API_KEY: # Nécessaire pour le LLM Juge par défaut
        logger.error("Clé API OpenAI non configurée. Impossible d'exécuter l'évaluation de la synthèse.")
        return

    # --- PRÉPARER VOS DONNÉES DE TEST ICI ---
    sample_query_synth = "What are the main challenges in robotic grasping using reinforcement learning?"
    sample_context_synth = """
    Challenge 1: Sample Inefficiency. RL algorithms often require a vast amount of data (trials) to learn effective grasping policies. This is costly and time-consuming on physical robots.
    Challenge 2: Sim-to-Real Gap. Models trained in simulation may not transfer well to real robots due to differences in dynamics, sensing, and appearance.
    Challenge 3: Reward Design. Crafting appropriate reward functions that guide the agent towards successful and robust grasping without unintended behaviors is difficult.
    Challenge 4: High-Dimensional State/Action Spaces. Grasping involves continuous and high-dimensional inputs (e.g., camera images) and outputs (e.g., robot joint commands).
    (Source: Fictional summary based on general knowledge for demo purposes)
    """
    # Cette synthèse est un exemple. Remplacez-la par une VRAIE sortie de votre SynthesisAgent.
    sample_synthesis_to_eval = """
    Robotic grasping using reinforcement learning faces several key challenges. Firstly, sample inefficiency means many trials are needed.
    Secondly, bridging the sim-to-real gap is problematic due to mismatches between simulation and reality.
    Thirdly, designing effective reward functions is complex. Lastly, the high dimensionality of state and action spaces poses difficulties.
    These challenges are actively being researched.
    """
    # -----------------------------------------

    # Initialiser le logger W&B
    wb_logger_synth = WandBMetricsLogger(
        project_name="CognitiveSwarm-Evaluations-Notebook",
        run_name=f"Synthesis_Eval_Demo_Judge_{judge_model_override or settings.DEFAULT_OPENAI_MODEL.replace('.', '_')}",
        config_to_log={
            "evaluation_type": "Synthesis",
            "original_query": sample_query_synth,
            "judge_llm": judge_model_override or settings.DEFAULT_OPENAI_MODEL,
            # On pourrait aussi logger un hash ou un ID du contexte/synthèse pour la traçabilité
        },
        tags=["synthesis", "notebook_demo", "llm_as_judge"]
    )
    if wb_logger_synth.is_disabled:
        logger.warning("Logging W&B désactivé. Les métriques de synthèse ne seront pas envoyées à W&B.")
    else:
        wb_logger_synth.start_run()
    
    evaluator_synth = None
    try:
        evaluator_synth = SynthesisEvaluator(judge_llm_model_name=judge_model_override)
    except Exception as e:
        logger.error(f"Échec d'initialisation du SynthesisEvaluator: {e}", exc_info=True)
        if wb_logger_synth and not wb_logger_synth.is_disabled: wb_logger_synth.end_run(exit_code=1)
        return

    logger.info(f"Évaluation de la synthèse pour la requête: '{sample_query_synth[:50]}...'")
    synthesis_metrics = await evaluator_synth.evaluate_synthesis(
        query=sample_query_synth,
        synthesis=sample_synthesis_to_eval,
        context=sample_context_synth
    )

    evaluator_synth.print_results(synthesis_metrics, query=sample_query_synth)

    if wb_logger_synth and not wb_logger_synth.is_disabled and wb_logger_synth.wandb_run:
        wb_logger_synth.log_synthesis_evaluation_results(synthesis_metrics, eval_name="Synthesis_Demo_Eval")
        
        # Logger les détails (requête, contexte, synthèse, scores, raisons) dans une table W&B
        synth_detail_data = [{
            "query": sample_query_synth,
            "context_snippet": sample_context_synth[:500] + "...", # Extrait du contexte
            "evaluated_synthesis": sample_synthesis_to_eval,
            "relevance_score": synthesis_metrics.get("relevance", {}).get("score") if synthesis_metrics.get("relevance") else None,
            "relevance_reasoning": synthesis_metrics.get("relevance", {}).get("reasoning") if synthesis_metrics.get("relevance") else None,
            "faithfulness_score": synthesis_metrics.get("faithfulness", {}).get("score") if synthesis_metrics.get("faithfulness") else None,
            "faithfulness_reasoning": synthesis_metrics.get("faithfulness", {}).get("reasoning") if synthesis_metrics.get("faithfulness") else None,
        }]
        try:
            details_df_synth = pd.DataFrame(synth_detail_data)
            wb_logger_synth.log_dataframe_as_table(details_df_synth, "Synthesis_Evaluation_Run_Detail")
        except ImportError:
             logger.warning("Pandas not installé, impossible de logger la table de détails de synthèse.")
        except Exception as e_df_s:
             logger.error(f"Erreur lors du logging de la table de détails de synthèse: {e_df_s}")


    if wb_logger_synth and not wb_logger_synth.is_disabled and wb_logger_synth.wandb_run:
        wb_logger_synth.end_run()

# Exécuter la démo d'évaluation de synthèse
asyncio.run(run_synthesis_evaluation_notebook_demo())

### 4. Utilisation du Script `scripts/run_evaluation.py`

Pour des évaluations plus complètes et automatisées, notamment sur des jeux de données plus larges, il est recommandé d'utiliser le script CLI `scripts/run_evaluation.py`.

Ce script orchestre les `RagEvaluator` et `SynthesisEvaluator`, gère le chargement des datasets depuis des fichiers, et intègre le logging W&B de manière configurable.

**Exemple de commande (à exécuter dans votre terminal, depuis la racine du projet) :**
```bash
python -m scripts.run_evaluation --eval_type all \
    --rag_dataset data/evaluation/rag_eval_dataset.json \
    --synthesis_dataset data/evaluation/synthesis_eval_dataset.json \
    --wandb_project "CognitiveSwarm-MainEvals" \
    --wandb_run_name "Full_Eval_Run_$(date +%Y%m%d_%H%M)" \
    --wandb_tags "full_eval,scheduled" \
    --log_level INFO

## Conclusion de l'Évaluation et du Logging

Ce notebook a montré comment :
- Utiliser `RagEvaluator` pour évaluer la performance de récupération.
- Utiliser `SynthesisEvaluator` avec un LLM comme juge pour évaluer la qualité des synthèses.
- Intégrer ces évaluations avec `WandBMetricsLogger` pour un suivi sur Weights & Biases.

L'évaluation continue est une partie essentielle du développement de systèmes LLM robustes. Les métriques collectées peuvent guider les améliorations des prompts, des stratégies RAG, des modèles LLM choisis, etc.

N'oubliez pas de consulter votre tableau de bord Weights & Biases pour visualiser les résultats loggés !